In [4]:
import warnings
import numpy as np
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_format = 'png' #retina
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as stats
from scipy.stats import skew,norm
from scipy.stats.stats import pearsonr
from IPython.display import display
pd.options.display.max_columns = None

ModuleNotFoundError: No module named 'missingno'

In [ ]:
df = pd.read_csv("cs1_part1_res.csv")

In [ ]:
df.head()

In [5]:
#visualize missing data
msno.matrix(df)

NameError: name 'msno' is not defined

In [ ]:
print("df size is : {}".format(df.shape))
missingdf = (df.isnull().sum() / len(df)) * 100
missingdf = missingdf.drop(missingdf[missingdf == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :missingdf})
missing_data.head(20)

# remove unnecessary features

In [ ]:
#remove the column PID, not usefull features
df = df.drop("pid",axis=1)

In [ ]:
#remove utilities because the data mostly is the same only 3 entry is different
df = df.drop(['utilities'], axis=1)

# input missing values

is added based on the description notes

In [ ]:
msno.matrix(df.select_dtypes(include='object'))

In [ ]:
#object type
df_objs = df.select_dtypes(include='object')
df_objs

In [ ]:
#i try to group them up into clusters with same values

#values "none"
for col in ('pool_qc','misc_feature','garage_type','alley','fence','fireplace_qu','garage_finish', 'garage_qual', 'garage_cond','mas_vnr_type','ms_subclass'):
    df[col] = df[col].fillna('None')

In [ ]:
# For below categorical basement-related features, NaN means that there is no basement.
for col in ('bsmt_qual', 'bsmt_cond', 'bsmt_exposure', 'bsmtfin_type_1', 'bsmtfin_type_2'):
    df[col] = df[col].fillna('None')

In [ ]:
#MSZoning classification : 'RL' is common
df['ms_zoning'] = df['ms_zoning'].fillna(df['ms_zoning'].fillna("RL"))

In [ ]:
df["functional"] = df["functional"].fillna("Typ")

In [ ]:
# Electrical standard  = SBrkr
df['electrical'] = df['electrical'].fillna(df['electrical'].fillna("SBrkr"))

In [ ]:
# KitchenQual
df['kitchen_qual'] = df['kitchen_qual'].fillna(df['kitchen_qual'].fillna("TA"))

In [ ]:
# Exterior1st and Exterior2nd
df['exterior_1st'] = df['exterior_1st'].fillna(df['exterior_1st'].fillna('Other'))
df['exterior_2nd'] = df['exterior_2nd'].fillna(df['exterior_2nd'].fillna('Other'))

In [ ]:
#SaleType
df['sale_type'] = df['sale_type'].fillna(df['sale_type'].fillna('Oth'))

In [ ]:
msno.matrix(df.select_dtypes(include='object'))

In [ ]:
#integer type
df_objs = df.select_dtypes(exclude='object')
df_objs

In [ ]:
msno.matrix(df.select_dtypes(exclude='object'))

In [ ]:
# Replacing missing data with 0 (Since No garage = no cars in such garage).
for col in ('garage_yr_blt', 'garage_area', 'garage_cars'):
    df[col] = df[col].fillna(0)

In [ ]:
# missing values are likely zero for having no basement 
for col in ('bsmtfin_sf_1', 'bsmtfin_sf_2', 'bsmt_unf_sf','total_bsmt_sf', 'bsmt_full_bath', 'bsmt_half_bath'):
    df[col] = df[col].fillna(0)

In [ ]:
df["mas_vnr_area"] = df["mas_vnr_area"].fillna(0)

In [ ]:
#Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
df["lot_frontage"] = df.groupby("neighborhood")["lot_frontage"].transform(lambda x: x.fillna(x.median()))

In [ ]:
#there are still missing values for lot_frontage
#fill it with median
df["lot_frontage"] = df["lot_frontage"].fillna(df['lot_frontage'].median())

# some numeric are actually categorical

In [ ]:
#dang need run this everytime i imported from csv...cos evertime imported in the astype changed back
# Transforming required numerical features to categorical

df['ms_subclass'] = df['ms_subclass'].astype(str)
df['overall_cond'] = df['overall_cond'].astype(str)
df['yr_sold'] = df['yr_sold'].astype(str)
df['mo_sold'] = df['mo_sold'].astype(str)

In [ ]:
df['lot_frontage'].isnull().values.any()

In [ ]:
msno.matrix(df.select_dtypes(exclude='object'))

In [ ]:
#list of features with missng values
listmiss = df.columns[df.isnull().any()].tolist()
listmiss
#df.columns[df.isnull().any()] #does the same 
#if empty..means no more 

In [ ]:
df.info()

# save the files

In [ ]:
df.to_csv("cs1_part2_res.csv",index=False)